In [3]:
import tensorrt as trt
import pycuda.autoinit
import pycuda.driver as cuda


def build_engine(model_file, max_ws=512*1024*1024, fp16=False):
    print("building engine")
    TRT_LOGGER = trt.Logger(trt.Logger.WARNING)
    # logging_manager = trt.LoggingManager()
    # debug_level = logging_manager.LEVEL.DEBUG
    # builder = trt.Builder(debug_level)
    builder = trt.Builder(TRT_LOGGER)
    # builder.fp16_enabled = True
    config = builder.create_builder_config()
    config.max_workspace_size = max_ws
    if fp16:
        config.flags |= 1 << int(trt.BuilderFlag.FP16)

    explicit_batch = 1 << int(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH)
    network = builder.create_network(explicit_batch)
    with trt.OnnxParser(network, TRT_LOGGER) as parser:
        with open(model_file, 'rb') as model:
            parsed = parser.parse(model.read())
            print("network.num_layers", network.num_layers)
            #last_layer = network.get_layer(network.num_layers - 1)
            #network.mark_output(last_layer.get_output(0))
            engine = builder.build_engine(network, config=config)
            return engine
engine = build_engine("mymodel.onnx")


with open('tensorrt.engine', 'wb') as f:
    f.write(bytearray(engine.serialize()))

building engine
[07/28/2023-17:50:55] [TRT] [W] onnx2trt_utils.cpp:374: Your ONNX model has been generated with INT64 weights, while TensorRT does not natively support INT64. Attempting to cast down to INT32.
network.num_layers 786


/tmp/ipykernel_13397/2811469131.py:15: DeprecationWarning: Use set_memory_pool_limit instead.
  config.max_workspace_size = max_ws
/tmp/ipykernel_13397/2811469131.py:27: DeprecationWarning: Use build_serialized_network instead.
  engine = builder.build_engine(network, config=config)
